In [57]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [58]:
# Define the image transformation functions
def flip(image):
    # Flip the image horizontally
    return cv2.flip(image, 1)

def rotate(image):
    # Rotate the image by 30 degrees
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 30, 1)
    return cv2.warpAffine(image, M, (cols, rows))

In [59]:
def histogram_equalization(img):
    return cv2.equalizeHist(img)

In [60]:
def resize_image(img):
    return cv2.resize(img , (4*124 , 4*64))

In [61]:
def shadow_remove(img):
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # rgb_planes = cv2.split(img)
    # result_norm_planes = []
    # for plane in rgb_planes:
    #     dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
    #     bg_img = cv2.medianBlur(dilated_img, 21)
    #     diff_img = 255 - cv2.absdiff(plane, bg_img)
    #     norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    #     result_norm_planes.append(norm_img)
    # shadowremov = cv2.merge(result_norm_planes)
    # # shadowremov [ shadowremov > 250] = 0
    # shadowremov = 255 - shadowremov
    # return shadowremov

    # # first part
    # black_indices = np.all(img < 50, axis=2)
    # img[black_indices] = [0, 0, 0]

    # # second part
    # red = img[:, :, 2]
    # green = img[:, :, 1]
    # blue = img[:, :, 0]

    # maxColor = np.argmax(img, axis=2)
    # # 0 -> blue
    # # 1 -> green
    # # 2 -> red

    # img[(maxColor == 0) & (blue - green > 50)] = [255, 0, 0]
    # img[(maxColor == 1) & (green - blue > 50)] = [0, 255, 0]
    # img[(maxColor != 0) & (maxColor != 1)] = [0, 0, 0]

    return img

In [62]:
def extract_hand_segment(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Calculate the average brightness of the image
    brightness = np.mean(hsv[:, :, 2])

    # Define the range of skin color based on image brightness
    lower_skin = np.array([0, 45, min(90, brightness - 40)], dtype=np.uint8)
    upper_skin = np.array([30, 255, min(255, brightness + 40)], dtype=np.uint8)

    # Create a mask using the skin color range
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Dilate the mask to remove small holes in the object
    kernel = np.ones((25, 25), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=25)

    # Apply Gaussian blur to the mask to remove noise
    mask = cv2.GaussianBlur(mask, (5, 5), 100)

    # Find the contours of the object in the mask
    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (which should be the hand)
    contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    largest_contour = max(contour_sizes, key=lambda x: x[0])[1]

    # Create a mask of the hand contour
    hand_mask = np.zeros_like(mask)
    cv2.drawContours(hand_mask, [largest_contour], 0, 255, cv2.FILLED)

    # Dilate the hand mask to fill in any gaps
    kernel = np.ones((20, 20), np.uint8)
    dilated_mask = cv2.dilate(hand_mask, kernel, iterations=10)

    # Apply erosion to reduce the size of the segmented area
    kernel = np.ones((25, 25), np.uint8)
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=35)

    # Apply the hand mask to the original image to extract the hand
    hand_segment = cv2.bitwise_and(image, image, mask=eroded_mask)

    # Remove shadows from the hand segment
    hand_segment_no_shadow = shadow_remove(hand_segment)

    # Create a mask of the hand region with no shadows and no background
    hand_mask_no_shadow = cv2.bitwise_and(hand_mask, cv2.bitwise_not(hand_segment_no_shadow))

    # Apply the hand mask with no shadows and no background to the original image
    hand_segment_final = cv2.bitwise_and(image, image, mask=hand_mask_no_shadow)

    # Return the segmented hand image with no shadows and no background
    return hand_segment_final

In [63]:
folder_path = "result"
cwd = os.getcwd()
full_path = os.path.join(cwd, folder_path)

for i in range(0, 6):
    for j in range(1, 6):
        # Read input image for men
        men_filename = 'images/{}_men ({}).JPG'.format(i, j)
        men_image = cv2.imread(men_filename)
        if men_image is not None:
            #men_final = extract_hand_segment(men_image)
            men_final = shadow_remove(men_image)
            men_modified_filename = f"modified_{i}_men_{j}.jpg"
            men_modified_path = os.path.join(full_path, men_modified_filename)
            cv2.imwrite(men_modified_path, men_final)

        # Read input image for women
        women_filename = 'images/{}_woman ({}).JPG'.format(i, j)
        women_image = cv2.imread(women_filename)
        if women_image is not None:
            #women_final = extract_hand_segment(women_image)
            women_final = shadow_remove(women_image)
            women_modified_filename = f"modified_{i}_woman_{j}.jpg"
            women_modified_path = os.path.join(full_path, women_modified_filename)
            cv2.imwrite(women_modified_path, women_final)

print(f"Modified images are saved in: {full_path}")


Modified images are saved in: d:\MOHAMED ENGINEERING\cmp\third year\second\Neural networks\Project\test\result
